#### Usage: python3 extract_people_df.py <matsim_output.xml>

### Cloe Cortes Balcells 13/03/2022
### EPFL TRANSP-OR lab, Lausanne, Switzerland

Extracts all information regarding individuals in the synthetic population from
the MATSIM output for Switzerland. This script processes the XML by chunks,
handling up to 10,000 individuals at a time to efficiently manage memory usage
and processing time.

In [1]:
import pandas as pd
from lxml import etree

def parse_xml_chunk(xml_content):
    # Parse the XML content
    tree = etree.fromstring(xml_content)
    
    # Initialize empty lists to store the data
    data = {
        'person_id': [],
        'plan_selected': [],
        'act_type': [],
        'x': [],
        'y': [],
        'start_time': [],
        'end_time': [],
        'leg_mode': []
    }
    
    # Process each person in the XML
    for person in tree.xpath('.//person'):
        person_id = person.attrib['id']
        plan_selected = person.xpath('.//plan/@selected')[0]
        
        acts = person.xpath('.//act')
        legs = person.xpath('.//leg')
        
        for act in acts:
            data['person_id'].append(person_id)
            data['plan_selected'].append(plan_selected)
            data['act_type'].append(act.attrib.get('type', ''))
            data['x'].append(act.attrib.get('x', ''))
            data['y'].append(act.attrib.get('y', ''))
            data['start_time'].append(act.attrib.get('start_time', ''))
            data['end_time'].append(act.attrib.get('end_time', ''))
            data['leg_mode'].append('')
        
        for i, leg in enumerate(legs):
            if i < len(data['leg_mode']) - 1:
                data['leg_mode'][i] = leg.attrib.get('mode', '')
    
    # Create a DataFrame
    df = pd.DataFrame(data)
    return df


In [2]:
def process_xml_in_chunks(file_path, chunk_size=10000):
    # Context for iteratively parsing the file
    context = etree.iterparse(file_path, events=('end',), tag='person')
    chunk = []
    data_frames = []
    
    for event, elem in context:
        # Convert the Element and its tree to a string
        xml_str = etree.tostring(elem, encoding='utf-8')
        chunk.append(xml_str)
        
        # Clear the element to free up memory
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
        
        # If chunk size is reached, parse it and then clear the chunk
        if len(chunk) == chunk_size:
            df = parse_xml_chunk(b'<root>' + b''.join(chunk) + b'</root>')
            data_frames.append(df)
            chunk = []  # reset chunk
    
    # Handle the last chunk
    if chunk:
        df = parse_xml_chunk(b'<root>' + b''.join(chunk) + b'</root>')
        data_frames.append(df)
    
    # Concatenate all chunks into a single DataFrame
    final_df = pd.concat(data_frames, ignore_index=True)
    return final_df


In [4]:
file_path = '/Users/cloe/Documents/GitHub/data_abm/C2SMART-Year3-Project/input/final_population.xml'
final_df = process_xml_in_chunks(file_path, chunk_size=10000)
print(final_df.head())


  person_id plan_selected   act_type                 x                 y  \
0         0           yes       Home  979229.529089607   196871.10471297   
1         0           yes  Secondary  984208.138769453   199047.45886273   
2         0           yes       Work  986767.068531842  213857.295077988   
3         0           yes  Secondary  987128.406291556  213396.819119905   
4         0           yes  Secondary  985037.372194942  214645.592776036   

  start_time  end_time leg_mode  
0             10:26:00     walk  
1   10:44:00   11:3:00     walk  
2   11:25:00  16:10:00       pt  
3   16:13:00  16:20:00       pt  
4   16:26:00  16:50:00     bike  


In [5]:
final_df.to_csv('/Users/cloe/Documents/GitHub/data_abm/C2SMART-Year3-Project/input/nyc_activities.csv.gz')

In [6]:
len(final_df['person_id'].unique())

389301

In [7]:
# vd_df=pd.read_csv('/Users/cloe/Documents/GitHub/data_abm/Data/1_Original/vaud_activities.csv.gz')
# vd_df

In [8]:
df_pop=pd.read_csv('/Users/cloe/Downloads/ModeUpdated-927-NewCoordinate.csv')
df_pop.rename(columns={'Person_ID': 'person_id', 'page': 'age', 'pschool': 'higher_education','rpworker': 'employment','home_x_new': 'home_x','home_y_new': 'home_y','rpsex':'gender'}, inplace=True)
df_pop.head(5)

,Trip_Number,person_id,geo,unique_id_in_geo,pid,sample_geo,age,higher_education,gender,employment,...,O_Y,D_X,D_Y,man,o_x_new,o_y_new,d_x_new,d_y_new,home_x,home_y
0,1,0,1,1,1,3810,10,17,1,1,...,-74.018108,40.713016,-74.000151,1,979229.529090,196871.104713,984208.138769,199047.458863,979229.52909,196871.104713
1,2,0,1,1,1,3810,10,17,1,1,...,-73.998226,40.753665,-73.990915,1,984741.798278,199140.003719,986767.068532,213857.295078,979229.52909,196871.104713
2,3,0,1,1,1,3810,10,17,1,1,...,-73.991374,40.752401,-73.989611,1,986639.956234,213276.900767,987128.406292,213396.819120,979229.52909,196871.104713
3,4,0,1,1,1,3810,10,17,1,1,...,-73.989905,40.755829,-73.997158,1,987047.014884,212832.094535,985037.372195,214645.592776,979229.52909,196871.104713
4,5,0,1,1,1,3810,10,17,1,1,...,-73.997341,40.750869,-73.990420,1,984986.687989,214128.603390,986904.323463,212838.636491,979229.52909,196871.104713


In [9]:
df_pop_subset = df_pop.groupby('person_id').first().reset_index()
print(df_pop_subset)

         person_id  Trip_Number   geo  unique_id_in_geo  pid  sample_geo  age  \
0                0            1     1                 1    1        3810   10   
1                1            8     1                 1    2        3810    9   
2                2           12     1                 2    1        3810   10   
3                3           17     1                 2    2        3810    9   
4                4           19     1                 3    1        3810   10   
...            ...          ...   ...               ...  ...         ...  ...   
8245617    8258548     30993242  1622              1652    2        3901   12   
8245618    8258549     30993244  1622              1653    1        3901   14   
8245619    8258550     30993246  1622              1653    2        3901   12   
8245620    8258551     30993248  1622              1654    1        3901   13   
8245621    8258552     30993250  1622              1654    2        3901   13   

         higher_education  

In [12]:
final_df['person_id'] = final_df['person_id'].astype(int)
df_pop_subset['person_id'] = df_pop_subset['person_id'].astype(int)
# Merging df1 and df2
df1 = pd.merge(final_df, df_pop_subset[['person_id', 'age', 'gender','higher_education','employment','home_x','home_y']], on='person_id', how='left')

In [25]:
mapping_dict = {
    2: 0,  # Map value 0 to 0
    1: 1   # Map value 1 to 1
}
# Apply the mapping using replace() method
df1['employed'] = df1['employment'].replace(mapping_dict)
mapping_dict_gender = {
    1: 0,  # Map value 0 to 0
    2: 1   # Map value 1 to 1 
}
# Apply the mapping using replace() method
df1['gender'] = df1['gender'].replace(mapping_dict_gender)
mapping_dict_2 = {
    15: 1,  # Map value 1 to 0
    16: 1,
    # For all other values, map them to 1
    **{value: 0 for value in df1['higher_education'].unique() if value != 0}
}
# Apply the mapping using replace() method
df1['higher_educ'] = df1['higher_education'].replace(mapping_dict_2)
age_ranges = {
    1: "0-4",
    2: "5-9",
    3: "10-14",
    4: "15-17",
    5: "18-19",
    6: "20",
    7: "21",
    8: "22-24",
    9: "25-29",
    10: "30-34",
    11: "35-39",
    12: "40-44",
    13: "45-49",
    14: "50-54",
    15: "55-59",
    16: "60-61",
    17: "62-64",
    18: "65-66",
    19: "67-69",
    20: "70-74",
    21: "75-79",
    22: "80-84",
    23: "85+"
}
df1['age'] = df1['age'].replace(age_ranges)

In [20]:
df1[df1['employment']==2.0]

,person_id,plan_selected,act_type,x,y,start_time,end_time,leg_mode,age,gender,higher_education,employment,home_x,home_y,employed,higher_educ
181,36,yes,Home,979702.630931755,197462.319300267,,12:0:00,,70-74,1.0,17.0,2.0,979702.630932,197462.319300,0.0,0.0
182,36,yes,Secondary,983317.87071995,203842.432762327,12:20:00,12:40:00,,70-74,1.0,17.0,2.0,979702.630932,197462.319300,0.0,0.0
183,36,yes,Home,979702.630931755,197462.319300267,,,,70-74,1.0,17.0,2.0,979702.630932,197462.319300,0.0,0.0
184,37,yes,Home,979110.815080106,197943.353172912,,10:15:00,,35-39,1.0,17.0,2.0,979110.815080,197943.353173,0.0,0.0
185,37,yes,Secondary,983294.398267236,206056.838419479,10:30:00,12:0:00,,35-39,1.0,17.0,2.0,979110.815080,197943.353173,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417610,8254238,yes,Secondary,957612.47824128,148987.423802298,12:10:00,12:20:00,,85+,2.0,17.0,2.0,943938.935794,135388.869238,0.0,0.0
1417611,8254238,yes,Secondary,1024570.49795084,209194.411265119,13:20:00,13:45:00,,85+,2.0,17.0,2.0,943938.935794,135388.869238,0.0,0.0
1417612,8254238,yes,Secondary,1020652.6151147,207584.430038875,14:0:00,16:30:00,,85+,2.0,17.0,2.0,943938.935794,135388.869238,0.0,0.0
1417613,8254238,yes,Secondary,1023505.26324686,207834.435096725,16:40:00,17:10:00,,85+,2.0,17.0,2.0,943938.935794,135388.869238,0.0,0.0


In [26]:
def correct_time(time_str):
    if pd.isna(time_str):
        return time_str  # Return NaN as is
    try:
        hours, minutes = map(int, time_str.split(':'))
        hours = hours % 24  # Correct hours to be within a 0-23 range
        corrected_time = f'{hours:02}:{minutes:02}'
        return corrected_time
    except ValueError:
        return time_str  # Return the original string if there's any error in splitting

# Correct 'start_time' and 'end_time' columns
df1['start_time'] = df1['start_time'].apply(correct_time)
df1['end_time'] = df1['end_time'].apply(correct_time)

# Optionally, check and display the first few rows to verify corrections
print(df1[['start_time', 'end_time']].head())


  start_time  end_time
0             10:26:00
1   10:44:00   11:3:00
2   11:25:00  16:10:00
3   16:13:00  16:20:00
4   16:26:00  16:50:00


In [27]:
df2=df1.dropna()
# len(df1['person_id'].unique())
# len(df2['person_id'].unique())
df2.rename(columns={'person_id': 'id','act_type':'type'}, inplace=True)

/var/folders/1_/2k1w1vfj6pg6ctyrym7wc2tc0000gp/T/ipykernel_42394/3433363205.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.rename(columns={'person_id': 'id','act_type':'type'}, inplace=True)


In [28]:
# # Subset where activity type is 'Secondary'
# trips_df = df2[df2['type'] == 'Secondary']

# # Subset where activity type is not 'Secondary'
# activities_df = df2[df2['type'] != 'Secondary']
df2['facility'] = pd.factorize(list(zip(df2['x'], 
                                                                  df2['y'], 
                                                                  df2['type'])))[0]

/var/folders/1_/2k1w1vfj6pg6ctyrym7wc2tc0000gp/T/ipykernel_42394/4212276649.py:6: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  df2['facility'] = pd.factorize(list(zip(df2['x'],
/var/folders/1_/2k1w1vfj6pg6ctyrym7wc2tc0000gp/T/ipykernel_42394/4212276649.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['facility'] = pd.factorize(list(zip(df2['x'],


In [ ]:
from datetime import timedelta

# Assume df is your DataFrame

# First, ensure the 'start_time' and 'end_time' are in the correct time format
activity_vaud['start_time'] = pd.to_datetime(activity_vaud['start_time'], format='%H:%M:%S', errors='coerce').dt.time
activity_vaud['end_time'] = pd.to_datetime(activity_vaud['end_time'], format='%H:%M:%S', errors='coerce').dt.time

# Function to apply the necessary changes for each group
def process_individual(group):
    # Check if the first row has NaN in 'start_time'
    if pd.isna(group.iloc[0]['start_time']):
        group.at[group.index[0], 'start_time'] = pd.to_datetime('00:00:00').time()
    
    # Ensure the second to last end_time is not NaT before proceeding with last row adjustments
    if not pd.isna(group.iloc[-2]['end_time']):
        last_end_time = group.iloc[-2]['end_time']
        # Calculate and set the start_time and end_time for the last row if needed
        if pd.isna(group.iloc[-1]['start_time']) or pd.isna(group.iloc[-1]['end_time']):
            calculated_start_time = (pd.Timestamp(last_end_time.strftime('%H:%M:%S')) + timedelta(minutes=10)).time()
            group.at[group.index[-1], 'start_time'] = calculated_start_time
            group.at[group.index[-1], 'end_time'] = pd.to_datetime('23:59:00').time()
    
    return group

# Apply the function to each group and recombine them into a single DataFrame
df2 = df2.groupby('id').apply(process_individual)

In [32]:
df2[['id','type','x','y','start_time','end_time','leg_mode']].to_csv('/Users/cloe/Documents/GitHub/data_abm/Data/1_Original/nyc_trips.csv.gz')
df2[['id','type','x','y','start_time','end_time','facility']].to_csv('/Users/cloe/Documents/GitHub/data_abm/Data/1_Original/nyc_activities.csv.gz')
df2[['id','age','higher_education','gender','employed','home_x','home_y']].to_csv('/Users/cloe/Documents/GitHub/data_abm/Data/1_Original/nyc_population.csv.gz')

In [31]:
df2

,id,plan_selected,type,x,y,start_time,end_time,leg_mode,age,gender,higher_education,employment,home_x,home_y,employed,higher_educ,facility
0,0,yes,Home,979229.529089607,196871.10471297,,10:26:00,walk,30-34,0.0,17.0,1.0,979229.529090,196871.104713,1.0,0.0,0
1,0,yes,Secondary,984208.138769453,199047.45886273,10:44:00,11:3:00,walk,30-34,0.0,17.0,1.0,979229.529090,196871.104713,1.0,0.0,1
2,0,yes,Work,986767.068531842,213857.295077988,11:25:00,16:10:00,pt,30-34,0.0,17.0,1.0,979229.529090,196871.104713,1.0,0.0,2
3,0,yes,Secondary,987128.406291556,213396.819119905,16:13:00,16:20:00,pt,30-34,0.0,17.0,1.0,979229.529090,196871.104713,1.0,0.0,3
4,0,yes,Secondary,985037.372194942,214645.592776036,16:26:00,16:50:00,bike,30-34,0.0,17.0,1.0,979229.529090,196871.104713,1.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417616,8254239,yes,Secondary,981578.708105579,164301.25818818,9:10:00,10:25:00,,35-39,0.0,17.0,1.0,944684.830768,137529.871980,1.0,0.0,693414
1417617,8254239,yes,Secondary,980983.348972371,171563.81832679,10:40:00,11:35:00,,35-39,0.0,17.0,1.0,944684.830768,137529.871980,1.0,0.0,693415
1417618,8254239,yes,Secondary,979699.943943856,168287.633141857,11:55:00,14:45:00,,35-39,0.0,17.0,1.0,944684.830768,137529.871980,1.0,0.0,647065
1417619,8254239,yes,Secondary,980714.110554378,173305.704034043,15:5:00,16:10:00,,35-39,0.0,17.0,1.0,944684.830768,137529.871980,1.0,0.0,693416


In [68]:
df2.to_csv('/Users/cloe/Documents/GitHub/data_abm/Data/1_Original/nyc_all.csv.gz')

In [30]:
df2

,id,plan_selected,type,x,y,start_time,end_time,leg_mode,age,gender,higher_education,employment,home_x,home_y,employed,higher_educ,facility
0,0,yes,Home,979229.529089607,196871.10471297,,10:26:00,walk,30-34,0.0,17.0,1.0,979229.529090,196871.104713,1.0,0.0,0
1,0,yes,Secondary,984208.138769453,199047.45886273,10:44:00,11:3:00,walk,30-34,0.0,17.0,1.0,979229.529090,196871.104713,1.0,0.0,1
2,0,yes,Work,986767.068531842,213857.295077988,11:25:00,16:10:00,pt,30-34,0.0,17.0,1.0,979229.529090,196871.104713,1.0,0.0,2
3,0,yes,Secondary,987128.406291556,213396.819119905,16:13:00,16:20:00,pt,30-34,0.0,17.0,1.0,979229.529090,196871.104713,1.0,0.0,3
4,0,yes,Secondary,985037.372194942,214645.592776036,16:26:00,16:50:00,bike,30-34,0.0,17.0,1.0,979229.529090,196871.104713,1.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417616,8254239,yes,Secondary,981578.708105579,164301.25818818,9:10:00,10:25:00,,35-39,0.0,17.0,1.0,944684.830768,137529.871980,1.0,0.0,693414
1417617,8254239,yes,Secondary,980983.348972371,171563.81832679,10:40:00,11:35:00,,35-39,0.0,17.0,1.0,944684.830768,137529.871980,1.0,0.0,693415
1417618,8254239,yes,Secondary,979699.943943856,168287.633141857,11:55:00,14:45:00,,35-39,0.0,17.0,1.0,944684.830768,137529.871980,1.0,0.0,647065
1417619,8254239,yes,Secondary,980714.110554378,173305.704034043,15:5:00,16:10:00,,35-39,0.0,17.0,1.0,944684.830768,137529.871980,1.0,0.0,693416
